<img src="images/datastaxdevs_banner.png" width="600" height="200">

# Algorithm 5: Collaborative Filtering
------
<img src="images/seinfeld.jpg" width="400" height="400">

#### Real Dataset: http://eigentaste.berkeley.edu/dataset/ Dataset 2 

## What are we trying to learn from this dataset?

### Can Collaborative Filtering be used to find which jokes to recommend to our users?

In [ ]:
import os
from pyspark.sql import SparkSession
from IPython.display import IFrame
#
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import explode, col
#
#
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
#
from dotenv import load_dotenv, find_dotenv

from tools import showDF, examineCassandraTable

In [ ]:
# read .env file for connection params
dotenv_file = find_dotenv('.env')
load_dotenv(dotenv_file)
astraUsername = os.environ['ASTRA_DB_CLIENT_ID']
astraPassword = os.environ['ASTRA_DB_CLIENT_SECRET']
astraSecureConnect = os.environ['ASTRA_DB_SECURE_BUNDLE_PATH']
astraKeyspace = os.environ['ASTRA_DB_KEYSPACE']

## Inspect input data: Table(s)

### Connect to Cassandra

In [ ]:
cloud_config = {
    'secure_connect_bundle': '/home/jovyan/' + astraSecureConnect
}
auth_provider = PlainTextAuthProvider(username=astraUsername, password=astraPassword)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

### Set keyspace 

In [ ]:
session.set_keyspace(astraKeyspace)

### Examine table `jokes` (structure and contents)

In [ ]:
print(examineCassandraTable(session, astraKeyspace, 'jokes'))

### What do these 3 columns represent: 

* **Column 1**: User id
* **Column 2**: Joke id
* **Column 3**: Rating of joke (-10.00 - 10.00) 

**Note**: the table contains a sample of 10'000 rows. The full dataset has over 1 _million_ rows.
<img src="images/laughing.gif" width="300" height="300">

# Machine Learning with Apache Cassandra & Apache Spark
<img src="images/sparklogo.png" width="150" height="200">

### Create a Spark session that is connected to the database. From there load each table into a Spark Dataframe and take a count of the number of rows in each.

In [ ]:
spark = SparkSession \
    .builder \
    .appName('demo') \
    .master('local') \
    .config( \
        'spark.cassandra.connection.config.cloud.path', \
        'file:' + '/home/jovyan/' + astraSecureConnect) \
    .config('spark.cassandra.auth.username', astraUsername) \
    .config('spark.cassandra.auth.password', astraPassword) \
    .getOrCreate()

jokeTable = spark \
    .read \
    .format('org.apache.spark.sql.cassandra') \
    .options(table='jokes', keyspace=astraKeyspace) \
    .load()

print ('Table Row Count:')
print (jokeTable.count())

### Split dataset into training and testing set 

In [ ]:
(training, test) = jokeTable.randomSplit([0.8, 0.2])

training_df = training.withColumn('rating', training.rating.cast('int'))
testing_df = test.withColumn('rating', test.rating.cast('int'))

showDF(training_df, limitRows=10)

## Setup for Collaborative Filtering with ALS ("alternating least squares")

https://spark.apache.org/docs/latest/ml-collaborative-filtering.html

In [ ]:
als = ALS(
    maxIter=5,
    regParam=0.2,
    userCol='userid',
    itemCol='jokeid',
    ratingCol='rating',
    coldStartStrategy='drop',
)

model = als.fit(training_df)

In [ ]:
# Evaluate the model by computing the RMSE on the test data
test_results = model.transform(testing_df)
showDF(test_results)

evaluator = RegressionEvaluator(
           metricName="rmse", 
           labelCol="rating", 
           predictionCol="prediction") 

RMSE = evaluator.evaluate(test_results)
print('RMSE = %.4f' % RMSE)

In [ ]:
# Generate top 10 joke recommendations for each user
userRecs = model.recommendForAllUsers(10)

showDF(userRecs, limitRows=10)

In [ ]:
# Generate top 10 user recommendations for each joke
jokeRecs = model.recommendForAllItems(10)

showDF(jokeRecs, limitRows=10)

#### Check recommendations for a single user

In [ ]:
# This is easier (for humans) than "showDF(userRecs.filter(userRecs.userid == 65), limitRows=10)"

showDF(
    userRecs.filter(userRecs.userid == 65) \
        .withColumn('recomm', explode('recommendations')) \
        .select('userid', col('recomm.jokeid'), col('recomm.rating')),
    limitRows=10,
)

In [ ]:
IFrame(src='images/init94.html', width=700, height=300)

In [ ]:
IFrame(src='images/init43.html', width=700, height=300)

#### Stop the Spark session

In [ ]:
spark.stop()